In [1]:
import run_simulation as rs
import numpy as np	
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import tdgl
import os 


c:\Program Files\Python310\lib\site-packages\cupyx\jit\_interface.py:173: FutureWarning: cupyx.jit.rawkernel is experimental. The interface can change in the future.
  cupy._util.experimental('cupyx.jit.rawkernel')


In [2]:
rs.check_system()

GPU is available
GPU:  b'NVIDIA GeForce RTX 4060 Laptop GPU'


In [4]:
xi=0.1
size=1.0
min_c=0.0
max_c=20
num_c=3
H_field_range=np.linspace(0,25,3)
current_range=np.linspace(min_c,max_c,num_c)
device=rs.define_4_terminal_mesh(size,size,xi=xi, probes=[((-size*(0.4),0),
					   				(size*(0.4),0),)],
									terminal_width=size/30,
									terminal_height=size/10
									)
z=np.zeros((10,10))

In [5]:
folder_path = 'H_fields'
files = rs.get_all_files(folder_path)

data_list=[]

for file in files:
	H_field, current = rs.extract_numbers(file)
	array=np.load(file)
	data=[H_field,current,array]
	data_list.append(data)
	

In [7]:
H_fields_list=[]
currents_for_H=[]
currents=[]


for data in data_list:
	if data[0] not in H_fields_list:
		H_fields_list.append(data[0])
		currents_for_H.append(currents)
		currents=[data[1]]
	else:
		index=H_fields_list.index(data[0])
		currents.append(data[1])
		
currents_for_H.append(currents)
currents_for_H=currents_for_H[1:]
	

found_Hs,missing_Hs=rs.check_elements_in_list(H_field_range,H_fields_list)

print(f"found data for H= {found_Hs}")
print(f"missing data for H= {missing_Hs}")

simulations_to_be_computed={}

for ii in range(len(found_Hs)):
	found_C,missing_C=rs.check_elements_in_list(current_range,currents_for_H[ii])

	simulations_to_be_computed[found_Hs[ii]]=np.array(missing_C)

for ii in range(len(missing_Hs)):

	simulations_to_be_computed[missing_Hs[ii]]=current_range

for key, value in simulations_to_be_computed.items():
    print(f"{key}: {value}")

found data for H= [0.0, 12.5, 25.0]
missing data for H= []
0.0: []
12.5: []
25.0: []


In [38]:
rs.build_mesh(device,max_edge_L=xi/4,plot=False)

Constructing Voronoi polygons: 100%|██████████| 5730/5730 [00:04<00:00, 1280.82it/s]

num_sites: 5730
num_elements: 11058
min_edge_length: 0.006176786351417163
max_edge_length: 0.02562758848145121
mean_edge_length: 0.014395335339315689
min_area: 2.309249471036603e-05
max_area: 0.0003865282402036166
mean_area: 0.00017452006980802793
coherence_length: 0.1
length_units: um


In [39]:
for H in simulations_to_be_computed:
	currents=simulations_to_be_computed[H]
	if currents.size == 0:
		continue
	else:
		print(f"simulating H={H}, c={currents}------------------------------------")
		solutions,currents=rs.current_series(device,
						current_range=currents,
						H_field=H,
						solve_time=50
						)
		jj=0
		for solution in solutions:
			voltage=solution.dynamics.voltage()
			time=solution.dynamics.time
			solution.to_hdf5(f'H_solutions/solution_{H}_{currents[jj]}_.hdf5')
			np.save(f'{folder_path}/voltages_{H}_{currents[jj]}_.npy',voltage)
			jj+=1


simulating H=12.5, c=[ 0. 10. 20.]------------------------------------


Simulating: 100%|█████████▉| 50/50 [00:19<00:00,  2.53tau/s ]


1/None done


Simulating: 100%|█████████▉| 50/50 [00:34<00:00,  1.46tau/s ]


2/None done


Simulating: 100%|█████████▉| 50/50 [00:28<00:00,  1.75tau/s ]


3/None done


--------------------------------------------------------------------------------


Simulating: 100%|█████████▉| 100/100 [00:21<00:00,  4.66tau/s ]


1/3 done


Simulating: 100%|█████████▉| 100/100 [00:42<00:00,  2.34tau/s ]


2/3 done


...

1/3 done


Simulating: 100%|█████████▉| 100/100 [00:50<00:00,  1.97tau/s ]


2/3 done


Simulating: 100%|█████████▉| 100/100 [00:49<00:00,  2.00tau/s ]


3/3 done
